In [ ]:
#################################################################### Loading packages ####################################################################
import sys as sys
import os
import numpy as np
import scipy.stats as scs
from matplotlib import pyplot as plt
from mpl_toolkits.basemap import Basemap
from tqdm import tqdm
import pickle
import math 
import re
from Censored_copula import *
from scipy.spatial import distance_matrix
from sklearn.metrics.pairwise import euclidean_distances 
import pandas as pd
from scipy.optimize import dual_annealing
from joblib import Parallel, delayed
import psutil
from scipy.spatial.distance import pdist
from sklearn.metrics import pairwise_distances
from sklearn.gaussian_process.kernels import Matern
plt.rcParams.update({'font.size': 30})

#################################################################### Loading GNM ####################################################################
model_version = 0 
target_distribution = "gamma_hurdle"
base_nn = "HConvLSTM_tdscale"
sys.path.append('../')

import pickle

path_ = f"Checkpoints/uk_rain_DGLM_{base_nn}_{target_distribution}/lightning_logs/version_{model_version}/"
#ckpt_path = glms.NeuralDGLM.get_ckpt_path(os.path.join(path_,"checkpoints"))
ckpt_path = 'Checkpoints/uk_rain_DGLM_HConvLSTM_tdscale_gamma_hurdle/lightning_logs/version_0/checkpoints/epoch=3-step=9008-val_loss_loss=-0.552-val_metric_mse_rain=26.500.ckpt'
print([path_,ckpt_path])
'''
scaler_features, scaler_target = glms.NeuralDGLM.load_scalers(path_)
model = glms.NeuralDGLM.load_from_checkpoint(ckpt_path, save_hparams=False, scaler_features=scaler_features, scaler_target=scaler_target)
model.freeze()
model.eval()
'''
GNM_output = pickle.load( open(os.path.join(path_,"test_output_1999-07_2019-07_test20y.pkl"),"rb") )

#################################################################### Problematic locations? ###############################################################

location_keys = list(GNM_output.keys())                 # '([0, 16], [44, 60])', ...
output_keys = list(GNM_output[location_keys[0]].keys()) # 0'pred_mu', 1'pred_disp', 2'target_did_rain', 3'target_rain_value', 4'date', 5'pred_p', 6'mask', 7'idx_loc_in_region'

# May have to remove a location that doesn't have the correct number of days:
location_keys.remove(location_keys[344]) #to remove for gamma_20_20_test
#location_keys.remove(location_keys[171]) # to remove for gamma_20_20_unseen
#location_keys.remove(location_keys[344]) # to remove for 10 y train
#location_keys.remove(location_keys[312])  # to remove for 2y_test
#location_keys.remove(location_keys[336]) #to remove for gamma_20_20 train  
#################################################################### Utility functions ####################################################################
def give_lat_lon(location_key):
    return [ int(re.findall(r'\d+',string)[0]) for string in location_key.split(',')]

def is_loc_land(location,row,column):
    return GNM_output[location]['mask'][0][row][column]

#location_keys.remove('lat_50.95_49.35_lon_-4.55_-2.95') # 'lat_50.95_49.35_lon_-4.55_-2.95' has no data after week indexed 255, so I did not consider it.
day_dates = [pd.to_datetime(i).date() for i in GNM_output[location_keys[0]]['date']]
output_keys.remove('idx_loc_in_region') # not usefull
output_keys.remove('date') # no more usefull

#################################################################### Making list for land locations ####################################################################
#GNM_landonly[location][param][day]
# 0'pred_mu', 1'pred_disp', 2'target_did_rain', 3'target_rain_value', 4'pred_p', 5'location'

GNM_landonly = [] 
land_counter = -1
for location in tqdm(location_keys):
    for col in range(4):
        for row in range(4):
            if not is_loc_land(location,row,col):#check if land
               continue  
            land_counter+=1
            GNM_landonly.append([[],[],[],[],[]]) # parameters except location
            temp_lat_lon = give_lat_lon(location)
            GNM_landonly[land_counter].append([temp_lat_lon[1]-8+row,(temp_lat_lon[2])+6+col]) #location
            for param_idx,param in enumerate(output_keys[:-1]):
                for day in range(len(GNM_output[location_keys[0]]['pred_mu'])):
                    qqq = [location,param,day,row,col]
                    GNM_landonly[land_counter][param_idx].append(GNM_output[location][param][day][row,col])

dist_mat = distance_matrix(np.matrix([[GNM_landonly[loc][5][0],GNM_landonly[loc][5][1]] for loc in range(len(GNM_landonly))]),np.matrix([[GNM_landonly[loc][5][0],GNM_landonly[loc][5][1]] for loc in range(len(GNM_landonly))]))

#################################################################### Elevation data ####################################################################
import netCDF4 as nc
fn = 'topo_0.1_degree.nc'
ds = nc.Dataset(fn)
h = ds['Band1'][:]
h = np.flip(np.matrix(h),axis=0)
topo_dist = euclidean_distances(np.array([h[loc[5][0],loc[5][1]+2] for loc in GNM_landonly]).reshape(-1,1))

In [ ]:
for i,k in enumerate(location_keys):
    if k=='([80, 96], [64, 80])':
        print(i,k)

# Warwing: check if the correct locations were removed

# Diagnostics

### Marginals

In [ ]:
# Draw time-series plot with quantiles for predicted vs trget rain at a given location. ~3s 
# Just change values at the top.

#ts_plot_location = 3055
for ts_plot_location in [954]:
    print(ts_plot_location)
    ts_plot_left_lim,ts_plot_right_lim = ts_plot_location+0,ts_plot_location+0+360

    #################################################################### Do not touch the rest ####################################################################
    import matplotlib.dates as mdates

    # mean and 68, 95 percentiles 
    g_median = []
    g_q25 = []
    g_q75 = []
    g_q68 = []
    g_q95 = []
    g_q32 = []
    g_q05 = []

    #GNM_landonly[location][param][day]
    ## 0'pred_mu', 1'pred_disp', 2'target_did_rain', 3'target_rain_value', 4'pred_p', 5'location'

    #gamma_outputs [place][parameter][day][row][column]
    # 0 'pred_mu', 1'pred_disp', 2'target_did_rain', 3'target_rain_value', 4'date', 5'pred_p', 6'location'.

    for m,d,i in zip([GNM_landonly[ts_plot_location][0][i] for i in range(len(GNM_landonly[0][0]))],[GNM_landonly[ts_plot_location][1][i] for i in range(len(GNM_landonly[0][0]))],range(len(GNM_landonly[0][0]))) :
        rv = scs.gamma(scale=m*d,a=1/d)  
        
        g_median.append(rv.ppf((GNM_landonly[ts_plot_location][4][i]-1+0.5)/GNM_landonly[ts_plot_location][4][i]))
        g_q25.append(rv.ppf((GNM_landonly[ts_plot_location][4][i]-1+0.25)/GNM_landonly[ts_plot_location][4][i]))
        g_q75.append(rv.ppf((GNM_landonly[ts_plot_location][4][i]-1+0.75)/GNM_landonly[ts_plot_location][4][i]))
        g_q95.append(rv.ppf((GNM_landonly[ts_plot_location][4][i]-1+0.95)/GNM_landonly[ts_plot_location][4][i]))
        g_q05.append(rv.ppf((GNM_landonly[ts_plot_location][4][i]-1+0.05)/GNM_landonly[ts_plot_location][4][i]))

    for q in [g_median,g_q25,g_q05,g_q32,g_q68,g_q75,g_q95]:
        for i in range(len(q)):
            if math.isnan(q[i]):
                q[i]=0
                
    plt.figure(figsize=(25, 8),dpi=700)
    plt.plot(day_dates,g_q05,color='skyblue',linewidth=0.6,alpha=1,label='forecast percentiles 05 to 95')
    plt.plot(day_dates,g_q25,color='navy',linewidth=0.6,alpha=1,label='forecast percentiles 25 to 75')
    plt.plot(day_dates,g_median,color='blue',linewidth=1,alpha=1,label='forcast median')
    plt.plot(day_dates,g_q75,color='navy',linewidth=0.6,alpha=1)
    plt.plot(day_dates,g_q95,color='skyblue',linewidth=0.6,alpha=1)
    plt.fill_between(day_dates, g_median, g_q75, where=([g_median[k] < g_q75[k] for k in range(len(GNM_landonly[ts_plot_location][4]))]), color='navy', alpha=0.6,interpolate=True)
    plt.fill_between(day_dates, g_median, g_q25, where=([g_median[k] > g_q25[k] for k in range(len(GNM_landonly[ts_plot_location][4]))]), color='navy', alpha=0.6,interpolate=True)
    plt.fill_between(day_dates, g_q95, g_q75, where=([g_q95[k] > g_q75[k] for k in range(len(GNM_landonly[ts_plot_location][4]))]), color='skyblue', alpha=0.6,interpolate=True)
    plt.fill_between(day_dates, g_q05, g_q25, where=([g_q05[k] < g_q25[k] for k in range(len(GNM_landonly[ts_plot_location][4]))]), color='skyblue', alpha=0.6,interpolate=True)
    plt.scatter(day_dates,[GNM_landonly[ts_plot_location][3][i] for i in range(len(GNM_landonly[0][0]))],color='r',marker='1',s=20,label='observed rain')
    plt.plot(day_dates,[GNM_landonly[ts_plot_location][3][i] for i in range(len(GNM_landonly[0][0]))],color='r',alpha=0.6,linewidth=0.3)
    plt.xlim((day_dates[ts_plot_left_lim],day_dates[ts_plot_right_lim]))
    plt.title('Gamma model at location '+str(GNM_landonly[ts_plot_location][5]))
    plt.xlabel('Day index')
    plt.ylabel('Precipitation (mm)')
    plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=14))
    plt.gcf().autofmt_xdate()
    plt.legend()
    plt.show()

In [ ]:
# Densities plots
# Just change values at the top.

density_plot_location = 111
density_plot_start,density_plot_end = 370,376

#################################################################### Do not touch the rest ####################################################################
import matplotlib

x=[i/20 for i in range(400)]
plt.figure(figsize=(10,(density_plot_end-density_plot_start)*2))
colors = matplotlib.cm.viridis(np.linspace(0, 1, len(range(density_plot_end,density_plot_start,-1))))
for idx,col in zip(reversed(range(density_plot_start-1,density_plot_end)),colors):
    rv=scs.gamma(scale=[GNM_landonly[density_plot_location][1][i] for i in range(len(GNM_landonly[ts_plot_location][4]))][idx]*[GNM_landonly[density_plot_location][0][i] for i in range(len(GNM_landonly[ts_plot_location][4]))][idx],a=1/[GNM_landonly[density_plot_location][1][i] for i in range(len(GNM_landonly[ts_plot_location][4]))][idx])
    level=(-0.3+(idx+1-density_plot_start)*0.25)
    plt.plot(x,level+rv.pdf(x),color='black',alpha=0.7)
    plt.fill_between(x,y1=level+rv.pdf(x),y2=level,where=([level+rv.pdf(x_)>level for x_ in x]),color=col,alpha=0.5)
    plt.text(15,level+0.03,'p= '+str(round([GNM_landonly[density_plot_location][4][i] for i in range(len(GNM_landonly[ts_plot_location][4]))][idx],2)))
    plt.scatter([GNM_landonly[density_plot_location][3][i] for i in range(len(GNM_landonly[ts_plot_location][4]))][idx],level,marker='v',color='b')
plt.xlabel('Precipitation (mm)')
plt.title('Gamma(y) in location '+str(GNM_landonly[density_plot_location][5]))
plt.yticks([-0.3+(idx-density_plot_start)*0.25 for idx in range(density_plot_end,density_plot_start,-1)],reversed([pd.to_datetime(i).date() for i in GNM_output[location_keys[0]]['date'][density_plot_start:density_plot_end]]))
#plt.yticks([-0.3+(idx-density_plot_start)*0.25 for idx in range(density_plot_end,density_plot_start,-1)],[k for k in range([density_plot_start:density_plot_end]])
plt.show()

### Simulations

In [ ]:
covariance_matrix = np.nan_to_num(Matern(length_scale=400,nu=3.5).__call__(0.9*dist_mat+((1-0.9)/70)*topo_dist))
sim_test = scs.multivariate_normal(mean=np.zeros(len(dist_mat)),cov= covariance_matrix ,allow_singular=True).rvs(size=100)

def GNM_x_to_y(x_one_day_all_locs,day_idx):
    m = np.array([GNM_landonly[l][0][day_idx] for l in range(len(x_one_day_all_locs))])
    d = np.array([GNM_landonly[l][1][day_idx] for l in range(len(x_one_day_all_locs))])
    p = np.array([GNM_landonly[l][4][day_idx] for l in range(len(x_one_day_all_locs))])
    sim_u = scs.norm.cdf(x_one_day_all_locs)
    censored_u = (sim_u+p-1)/p
    return np.nan_to_num(scs.gamma(a=1/d,scale=m*d).ppf(censored_u),nan=0)
    

In [ ]:
day_subset=np.random.choice(range(len(GNM_landonly[0][0])),50,replace=False)

'''# For robustness only
with open('day_subset_robustness.txt','rb') as f:
    day_subset = pickle.load(f)'''

sims_all_copula = []
for d in tqdm(day_subset):
    sims_all_copula.append([GNM_x_to_y([sim_test[s][l] for l in range(len(GNM_landonly))],d) for s in range(100)])

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time
from PIL import Image

day_subsample_idx = 32

min_rain = 0
max_rain = max([GNM_landonly[l][3][day_subset[day_subsample_idx]] for l in range(len(GNM_landonly))]) + 5

# Create a list to store frames
frames = []

# Set the figure size outside the loop
fig = plt.figure(figsize=(13, 9))

def plot_graphs(plots_nb):
    for sim in range(plots_nb):
        plt.clf()  # Clear the previous plot
        plt.xlim(0, 140)
        plt.ylim(-100, 0)

        plt.scatter([GNM_landonly[loc][5][1] for loc in range(len(sim_test[0]))],
                    [-GNM_landonly[loc][5][0] for loc in range(len(sim_test[0]))],
                    c=sims_all_copula[day_subsample_idx][sim],
                    cmap='inferno_r', s=27, marker='s', vmin=min_rain, vmax=max_rain)
        
        plt.xticks([])
        plt.yticks([])
        plt.title('Simulated rain sample ' + str(sim))

        # Convert the current plot to an image
        fig.canvas.draw()
        img = Image.frombytes('RGB', fig.canvas.get_width_height(), fig.canvas.tostring_rgb())
        frames.append(img)

        clear_output(wait=True)  # Clear the output for the next plot
        time.sleep(1)  # Pause for 1 second

# Call the function to create frames
plot_graphs(100)

# Save the frames as a GIF
frames[0].save('slideshow.gif', save_all=True, append_images=frames[1:], duration=1000, loop=0)


fig = plt.figure(figsize=(13, 9))
day_index = day_subset[day_subsample_idx]
plt.xlim(0,140)
plt.ylim(-100,0)
plt.scatter([GNM_landonly[loc][5][1] for loc in range(len(sim_test[0]))],[-GNM_landonly[loc][5][0] for loc in range(len(sim_test[0]))],c=[GNM_landonly[l][3][day_index] for l in range(len(GNM_landonly))][:len(sim_test[0])],cmap='inferno_r',s=27,marker='s',vmin=min_rain,vmax=max_rain)
plt.colorbar(location='bottom',fraction=0.0652,pad=0.001)
plt.title('Observed rain')
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:

for d in range(50):
    day_index = day_subset[d]
    fig = plt.gcf() 
    fig.set_size_inches(14,10)
    plt.xlim(0,140)
    plt.ylim(-100,0)
    plt.tight_layout()
    plt.scatter([GNM_landonly[loc][5][1] for loc in range(len(sim_test[0]))],[-GNM_landonly[loc][5][0] for loc in range(len(sim_test[0]))],c=[GNM_landonly[l][3][day_index] for l in range(len(GNM_landonly))][:len(sim_test[0])],cmap='inferno_r',s=27,marker='s')
    plt.colorbar(location='bottom',fraction=0.0652,pad=0.001)
    plt.title('obs'+ str(d))
    plt.xticks([])
    plt.yticks([])
    plt.show()

In [ ]:
#plot sampled normals uncensored or cenored and transformed
day_index = day_subset[40]

fig = plt.gcf() 
fig.set_size_inches(14,10)
plt.xlim(0,140)
plt.ylim(-100,0)
plt.tight_layout()
plt.scatter([GNM_landonly[loc][5][1] for loc in range(len(sim_test[0]))],[-GNM_landonly[loc][5][0] for loc in range(len(sim_test[0]))],c=[GNM_landonly[l][3][day_index] for l in range(len(GNM_landonly))][:len(sim_test[0])],cmap='inferno_r',s=27,marker='s')
plt.colorbar(location='bottom',fraction=0.0652,pad=0.001)
plt.title('obs')
plt.xticks([])
plt.yticks([])
plt.show()

min_rain = 0
max_rain = max([GNM_landonly[l][3][day_index] for l in range(len(GNM_landonly))])+2

fig = plt.gcf() 
fig.set_size_inches(14,10)
plt.tight_layout()
plt.xlim(0,140)
plt.ylim(-100,0)
plt.scatter([GNM_landonly[loc][5][1] for loc in range(len(sim_test[0]))],[-GNM_landonly[loc][5][0] for loc in range(len(sim_test[0]))],c=np.mean([GNM_x_to_y(sim_test[s],day_index) for s in range(25)],axis=0),cmap='inferno_r',s=27,marker='s',vmin=min_rain,vmax=max_rain)
plt.colorbar(location='bottom',fraction=0.0652,pad=0.001)
plt.title('median pred rain')
plt.xticks([])
plt.yticks([])
plt.show()

for idx,sim in enumerate(sim_test[5:43]):
    plt.xlim(0,140)
    plt.ylim(-100,0)
    fig = plt.gcf() 
    fig.set_size_inches(14,10)
    plt.tight_layout()

    plt.scatter([GNM_landonly[loc][5][1] for loc in range(len(sim_test[0]))],[-GNM_landonly[loc][5][0] for loc in range(len(sim_test[0]))],c=GNM_x_to_y(sim,day_index),cmap='inferno_r',s=27,marker='s',vmin=min_rain,vmax=max_rain)
    plt.colorbar(location='bottom',fraction=0.0652,pad=0.001)
    plt.xticks([])
    plt.yticks([])
    plt.title('sims '+str(idx))
    plt.show()

### Median forecast

In [ ]:
day_subset=np.random.choice(range(len(GNM_landonly[0][0])),500,replace=False)
#for comparison of marginals gamma vs CP
with open('day_subset_marginals.txt','rb') as f:
    day_subset = pickle.load(f)

In [ ]:

g_median_all= np.nan_to_num(
    scs.gamma(
        a=np.array([[GNM_landonly[l][0][d] for l in range(len(GNM_landonly))] for d in tqdm(range(len(GNM_landonly[0][0])))]) , 
        scale=np.reciprocal(np.array([[GNM_landonly[l][1][d] for l in range(len(GNM_landonly))] for d in range(len(GNM_landonly[0][0]))])) 
        ).ppf(
            (np.array([[GNM_landonly[l][4][d] for l in range(len(GNM_landonly))] for d in range(len(GNM_landonly[0][0]))])-0.5
            )/(
                np.array([[GNM_landonly[l][4][d] for l in range(len(GNM_landonly))] for d in tqdm(range(len(GNM_landonly[0][0])))])
                )
            ), 
            nan=0,posinf=20,neginf=0)


In [ ]:
median_x_all = [np.median([sim_test[s] for s in range(25)],axis=0) for d in tqdm(day_subset)]

In [ ]:
g_median_pred_all = []
for i,d in tqdm(enumerate(day_subset)):
    g_median_pred_all.append(GNM_x_to_y(median_x_all[i],d) )

In [ ]:
for i,m in enumerate(day_subset[:3]):
    min_rain = min([GNM_landonly[l][3][m] for l in range(len(GNM_landonly))])
    max_rain = max([GNM_landonly[l][3][m] for l in range(len(GNM_landonly))])

    fig = plt.gcf() 
    fig.set_size_inches(14,10)
    plt.xlim(0,140)
    plt.ylim(-100,0)
    plt.scatter([GNM_landonly[loc][5][1] for loc in range(len(sim_test[0]))],[-GNM_landonly[loc][5][0] for loc in range(len(sim_test[0]))],c=[GNM_landonly[l][3][m] for l in range(len(GNM_landonly))][:len(sim_test[0])],cmap='inferno_r',s=27,marker='s')
    plt.colorbar(location='bottom',fraction=0.0652,pad=0.001)
    plt.title('observed rain' + str(m))
    plt.show()

    fig = plt.gcf() 
    fig.set_size_inches(14,10)
    plt.xlim(0,140)
    plt.ylim(-100,0)
    plt.scatter([GNM_landonly[loc][5][1] for loc in range(len(sim_test[0]))],[-GNM_landonly[loc][5][0] for loc in range(len(sim_test[0]))],c=g_median_pred_all[i],cmap='inferno_r',s=27,marker='s',vmin=min_rain,vmax=max_rain)
    plt.colorbar(location='bottom',fraction=0.0652,pad=0.001)
    plt.title('median pred rain'+ str(m))
    plt.show()

    fig = plt.gcf() 
    fig.set_size_inches(14,10)
    plt.xlim(0,140)
    plt.ylim(-100,0)
    plt.scatter([GNM_landonly[loc][5][1] for loc in range(len(sim_test[0]))],[-GNM_landonly[loc][5][0] for loc in range(len(sim_test[0]))],c=g_median_all[m],cmap='inferno_r',s=27,marker='s',vmin=min_rain,vmax=max_rain)
    plt.colorbar(location='bottom',fraction=0.0652,pad=0.001)
    plt.title('median GNM rain'+ str(m))
    plt.show()

In [ ]:
with open('topo_dist.txt','wb') as f:
    pickle.dump(topo_dist,f)

In [ ]:
# Median rain forecasts for all days and locations
#GNM_landonly[location][param][day]


# Used to get the vectors for all locations and all times, not needed after that.

g_median_all=[]

for loc in tqdm(range(len(GNM_landonly))):
    for day in range(len(GNM_landonly[0][1])):
        m,d = GNM_landonly[loc][0][day] , GNM_landonly[loc][1][day]
        rv = scs.gamma(scale=m*d,a=1/d)  
        g_median_all.append(rv.ppf((GNM_landonly[loc][4][day]-1+0.5)/GNM_landonly[loc][4][day]))

g_median_all = np.nan_to_num(g_median_all,copy=True)

with open('g_median_all_20y_20y.txt','wb') as f:
    pickle.dump(g_median_all,f)

'''
with open('g_median_all_20y_20y.txt','rb') as f:
    g_median_all = pickle.load(f)
g_median_all_test = np.nan_to_num(g_median_all,nan=0)
'''

In [ ]:
def get_median(locs,nb_days):
    g_median_all_=[]
    for loc in locs:
        for day in range(nb_days):
            m,d = GNM_landonly[loc][0][day] , GNM_landonly[loc][1][day]
            rv = scs.gamma(scale=m*d,a=1/d)  
            g_median_all_.append(rv.ppf((GNM_landonly[loc][4][day]-1+0.5)/GNM_landonly[loc][4][day]))

    

    return g_median_all_

In [ ]:
def get_median(locs,nb_days):
    g_median_all_=[]
    for loc in locs:
        for day in range(nb_days):
            g_median_all_.append((GNM_landonly[loc][4][day]-1+0.5)/GNM_landonly[loc][4][day])

    return g_median_all_

In [ ]:
from joblib import Parallel, delayed
import psutil

size = math.ceil(len(GNM_landonly)/psutil.cpu_count())
locs_chuncks = [[i for i in range(len(GNM_landonly))][x:x+size] for x in range(0,len(GNM_landonly), size)]
print(size,psutil.cpu_count(),len(locs_chuncks))
list_g_median_all = Parallel(n_jobs=psutil.cpu_count())(delayed(get_median)(chunck[:2],2) for chunck in tqdm(locs_chuncks))

### ROC

In [ ]:
# getting vector of 'did it rain this much'
g_0mm=[]
g_5mm=[]
g_10mm=[]
g_15mm=[]
g_20mm=[]
g_25mm=[]

#GNM_landonly[location][param][day]
# 0'pred_alpha', 1'pred_beta', 2 rain>0 , 3'target_rain', 4 pred_p, 5 loc

def fast_roc(day,x):
   
   out = 1-(
         (1-np.array([GNM_landonly[l][4][day] for l in range(len(GNM_landonly))])) + 
         np.multiply(
            [GNM_landonly[l][4][day] for l in range(len(GNM_landonly))] , 
            scs.gamma( scale=[1/GNM_landonly[l][1][day] for l in range(len(GNM_landonly))] , 
                        a=[GNM_landonly[l][0][day] for l in range(len(GNM_landonly))]
               ).cdf(x)
            )        
      )

   return out


for d in tqdm(day_subset):
   g_0mm.append(fast_roc(d,0.01))
   g_5mm.append(fast_roc(d,5))
   g_10mm.append(fast_roc(d,10))
   g_15mm.append(fast_roc(d,15))
   g_20mm.append(fast_roc(d,20))
   g_25mm.append(fast_roc(d,25))

g_0mm_did_rain=[]
g_5mm_did_rain=[]
g_10mm_did_rain=[]
g_15mm_did_rain=[]
g_20mm_did_rain=[]
g_25mm_did_rain=[]
def roc_didrain(day,x):
    return np.array([GNM_landonly[l][3][day] for l in range(len(GNM_landonly))])>x
for d in tqdm(day_subset):
   g_0mm_did_rain.append(roc_didrain(d,0.01))
   g_5mm_did_rain.append(roc_didrain(d,5))
   g_10mm_did_rain.append(roc_didrain(d,10))
   g_15mm_did_rain.append(roc_didrain(d,15))
   g_20mm_did_rain.append(roc_didrain(d,20))
   g_25mm_did_rain.append(roc_didrain(d,25))

In [ ]:

with open('roc_convcnp20ytest.txt','rb') as f:
    [fpr_0mm, tpr_0mm,fpr_5mm, tpr_5mm,fpr_10mm, tpr_10mm,fpr_15mm, tpr_15mm,fpr_20mm, tpr_20mm,fpr_25mm, tpr_25mm,auc0,auc5,auc10,auc15,auc20,auc25] = pickle.load(f)

In [ ]:
# Drawing uniformly from marginals to compare vs CP
def GNM_unif_to_y(unif,day_idx):
    m = np.array([GNM_landonly[l][0][day_idx] for l in range(len(GNM_landonly))])
    d = np.array([GNM_landonly[l][1][day_idx] for l in range(len(GNM_landonly))])
    p = np.tile(np.array([GNM_landonly[l][4][day_idx] for l in range(len(GNM_landonly))]),(unif.shape[0],1))
    return np.nan_to_num(scs.gamma(a=1/d,scale=m*d).ppf(np.divide(unif+p-1,p)),nan=0)

sims_all_copula = []
for day in tqdm(day_subset):
    sims_all_copula.append(GNM_unif_to_y(np.random.uniform(size=(100,len(GNM_landonly))),day)) # HERE IS NOT COPULA BUT JUST UNIFORM

In [ ]:
# ROC for samples
sims_all_copula = np.array(sims_all_copula)
g_0mm=np.mean(sims_all_copula>0,axis=1).flatten()
g_5mm=np.mean(sims_all_copula>5,axis=1).flatten()
g_10mm=np.mean(sims_all_copula>10,axis=1).flatten()
g_15mm=np.mean(sims_all_copula>15,axis=1).flatten()
g_20mm=np.mean(sims_all_copula>20,axis=1).flatten()
g_25mm=np.mean(sims_all_copula>25,axis=1).flatten()
all_target_rain = np.array([np.array([GNM_landonly[loc][3][day] for day in day_subset])for loc in range(len(GNM_landonly))]).flatten().reshape(len(GNM_landonly),len(day_subset))
g_0mm_did_rain=(all_target_rain.transpose()>0).flatten()
g_5mm_did_rain=(all_target_rain.transpose()>5).flatten()
g_10mm_did_rain=(all_target_rain.transpose()>10).flatten()
g_15mm_did_rain=(all_target_rain.transpose()>15).flatten()
g_20mm_did_rain=(all_target_rain.transpose()>20).flatten()
g_25mm_did_rain=(all_target_rain.transpose()>25).flatten()

In [ ]:
# Restructuring data for ROC ~1m+3m/20y , 6m/30y
from sklearn import metrics
# g_25mm[loc][parallel run][0:rained more than x, 1:P it rained x][days]
roc_0_tf = np.array(g_0mm_did_rain).flatten()
roc_0_p = np.array(g_0mm).flatten()

roc_5_tf = np.array(g_5mm_did_rain).flatten()
roc_5_p = np.array(g_5mm).flatten()

roc_10_tf = np.array(g_10mm_did_rain).flatten()
roc_10_p = np.array(g_10mm).flatten()

roc_15_tf = np.array(g_15mm_did_rain).flatten()
roc_15_p = np.array(g_15mm).flatten()

roc_20_tf = np.array(g_20mm_did_rain).flatten()
roc_20_p = np.array(g_20mm).flatten()

roc_25_tf = np.array(g_25mm_did_rain).flatten()
roc_25_p = np.array(g_25mm).flatten()

###

fpr_0mm, tpr_0mm, _ = metrics.roc_curve(roc_0_tf,roc_0_p)
auc0 = metrics.roc_auc_score(roc_0_tf,roc_0_p)

fpr_5mm, tpr_5mm, _ = metrics.roc_curve(roc_5_tf,roc_5_p)
auc5 = metrics.roc_auc_score(roc_5_tf,roc_5_p)

fpr_10mm, tpr_10mm, _ = metrics.roc_curve(roc_10_tf,roc_10_p)
auc10 = metrics.roc_auc_score(roc_10_tf,roc_10_p)

fpr_15mm, tpr_15mm, _ = metrics.roc_curve(roc_15_tf,roc_15_p)
auc15 = metrics.roc_auc_score(roc_15_tf,roc_15_p)

fpr_20mm, tpr_20mm, _ = metrics.roc_curve(roc_20_tf,roc_20_p)
auc20 = metrics.roc_auc_score(roc_20_tf,roc_20_p)

fpr_25mm, tpr_25mm, _ = metrics.roc_curve(roc_25_tf,roc_25_p)
auc25 = metrics.roc_auc_score(roc_25_tf,roc_25_p)

In [ ]:
# ROC plot ~3m
plt.figure(figsize=(11,10))
plt.plot(fpr_0mm,tpr_0mm,label='AUC='+str(auc0),color='black',alpha=0.7)

plt.plot(fpr_5mm,tpr_5mm,label='AUC='+str(auc5),color='red',alpha=0.7)

plt.plot(fpr_10mm,tpr_10mm,label='AUC='+str(auc10),color='orange',alpha=0.7,linestyle='dashed')

plt.plot(fpr_15mm,tpr_15mm,label='AUC='+str(auc15),color='green',alpha=0.7,linestyle='dashdot')

plt.plot(fpr_20mm,tpr_20mm,label='AUC='+str(auc20),color='blue',alpha=0.7,linestyle='dotted')

plt.plot(fpr_25mm,tpr_25mm,label='AUC='+str(auc25),color='purple',alpha=0.7)

plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(['0mm AUC: '+str(round(auc0,5)),'5mm AUC:'+str(round(auc5,5)),'10mm AUC:'+str(round(auc10,5)),'15mm AUC:'+str(round(auc15,5)),'20mm AUC:'+str(round(auc20,5)),'25mm AUC:'+str(round(auc25,5))])
plt.tight_layout()
plt.savefig('ROC_GNM_20y_500subdays_test_vs_marginalCP.eps', format='eps')
#plt.title('Gamma Model United Kingdom')
#plt.savefig('ROC Gamma model UK')

### 1 - ecdf

In [ ]:
# P(rain>x) ~1m15 now. depending on samples and days used. 
from statsmodels.distributions.empirical_distribution import ECDF

# get empirical cdfs for median, sampled and target rain.
#gamma_ecdf = ECDF(np.array(g_median_pred_all).flatten())
target_ecdf = ECDF(np.array([np.array([GNM_landonly[loc][3][day] for loc in range(len(GNM_landonly))]) for day in tqdm(range(len(GNM_landonly[0][0])))]).flatten())
gamma_sampled_ecdf = ECDF(np.array(sims_all_copula).flatten())

#plt.plot(gamma_ecdf.x,1-gamma_ecdf.y,label='Median forecast')

plt.figure(figsize=(10,7))
plt.plot(gamma_sampled_ecdf.x[::1000],1-gamma_sampled_ecdf.y[::1000],linestyle='--',linewidth=5, zorder=5,label='Sampled forecasts')
#plt.scatter(gamma_sampled_ecdf.x[::10000000],1-gamma_sampled_ecdf.y[::10000000],linestyle='dotted',color='cornflowerblue',marker='x',linewidth=20,s=3)

plt.plot(target_ecdf.x[::100],1-target_ecdf.y[::100],linewidth=3,label='Target rainfall',color='black')
plt.xlabel('Precipitation (mm)')
plt.ylabel('Probability of precipitatoin > x')
plt.xlim(-1,25)
plt.legend()

### RMSB MAB

In [ ]:
# RMSB and MAB metrics across all days and locations ~4.9s
all_target_rain = np.array([np.array([GNM_landonly[loc][3][day]for day in range(len(GNM_landonly[0][0]))])for loc in range(4087)]).flatten().reshape(4087,len(GNM_landonly[0][0])) # [loc,day]
RMSB = math.sqrt(np.mean(np.power(np.subtract(all_target_rain,np.array(g_median_all).reshape((4087,len(GNM_landonly[0][0])))),2)))
MAB = np.mean(np.abs(all_target_rain-np.array(g_median_all).reshape((4087,len(GNM_landonly[0][0])))))
print('MAB = '+str(MAB)+', RMSB = '+str(RMSB))

In [ ]:
# Plot RMSB and MAB across time, averaged over all locations. 
import matplotlib.dates as mdates
plt.figure(figsize=(25,8),dpi=700)

RMSB_matrix = np.power(np.subtract(all_target_rain,np.array(g_median_all).reshape((4087,len(GNM_landonly[0][0])))),2)
MAB_matrix = np.abs(all_target_rain-np.array(g_median_all).reshape((4087,len(GNM_landonly[0][0]))))
RMSB_ts = [math.sqrt(np.mean(RMSB_matrix[:,day])) for day in range(len(GNM_landonly[0][0]))]
MAB_ts = [np.mean(MAB_matrix[:,day]) for day in range(len(GNM_landonly[0][0]))]
plt.plot(day_dates,RMSB_ts,alpha=0.6,label='RMSB')
plt.plot(day_dates,MAB_ts,alpha=0.6,label='MAB')
plt.legend()

plt.ylabel('Precipitation bias (mm)')
plt.xlabel('Date')
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=round(len(GNM_landonly[0][0])/15)))
plt.gcf().autofmt_xdate()
plt.show()

In [ ]:
plt.hist(RMSB_ts)
plt.title('RMSB across days')
plt.show()
plt.hist(MAB_ts)
plt.title('MAB across days')
plt.show()

### Bias Maps

In [ ]:
# Bias maps 2*2
test_arr = np.array([np.mean(sampled_rain_flatten_all.reshape(3,53118)[:,column]) for column in range(53118)]).reshape(13,4086)

MAB_map = [np.mean(MAB_matrix[loc,:]) for loc in range(4086)]
RMSB_map = [math.sqrt(np.mean(RMSB_matrix[loc,:])) for loc in range(4086)]
Bias_map = np.matrix(np.array(g_median_all).reshape((4087,2000))-all_target_rain).mean(1).tolist()[:-1]
day_subset = np.random.choice(range(2000),50)
Sampled_bias = np.divide(np.array([np.mean(test_arr[:,loc]) for loc in range(4086)])-np.array([np.mean(np.transpose(all_target_rain[loc,day_subset])) for loc in range(4086)]), # difference 
np.array([np.mean(np.transpose(all_target_rain[loc,day_subset])) for loc in range(4086)])) # normalize by average target

fig, axes = plt.subplots(nrows=2, ncols=2,sharex=True,sharey=True,figsize=(12.5,8.5))
for day,ax in tqdm(enumerate(axes.flat)):
    
    fig.set_dpi(500)
    ax.set_aspect('equal')
    ax.set_xticks([])
    ax.set_yticklabels([])
    plt.subplots_adjust(wspace=0, hspace=0)
    if day==0:
        #ax.set_title('MAB of median', y=1.0, pad=-10,size=10)
        im=ax.scatter([GNM_landonly[loc][5][1] for loc in range(4086)],[-GNM_landonly[loc][5][0] for loc in range(4086)],c=MAB_map,s=3.3,marker='s')
        fig.colorbar(im,ax=ax,location='left',fraction=0.03,pad=0.001).ax.tick_params(labelsize=3) 
        
    elif day==1:
        #ax.set_title('RMSB of median', y=1.0, pad=-10,size=10)
        im=ax.scatter([GNM_landonly[loc][5][1] for loc in range(4086)],[-GNM_landonly[loc][5][0] for loc in range(4086)],c=RMSB_map,s=3.3,marker='s')
        fig.colorbar(im,ax=ax,location='left',fraction=0.03,pad=0.001)
    elif day==2:
        #ax.set_title('Bias of median', y=1.0, pad=-10,size=10)
        im=ax.scatter([GNM_landonly[loc][5][1] for loc in range(4086)],[-GNM_landonly[loc][5][0] for loc in range(4086)],c=Bias_map,cmap='BrBG',s=3.3,marker='s',vmin=-4,vmax=4)
        fig.colorbar(im,ax=ax,location='left',fraction=0.03,pad=0.001)
    else:
        #ax.set_title('Bias of 10 averaged samples', y=1.0, pad=-10,size=10)
        im=ax.scatter([GNM_landonly[loc][5][1] for loc in range(4086)],[-GNM_landonly[loc][5][0] for loc in range(4086)],c=Sampled_bias,cmap='BrBG',s=3.3,marker='s',vmin=-5,vmax=5)
        fig.colorbar(im,ax=ax,location='left',fraction=0.03,pad=0.001)

plt.show()

In [ ]:

for map_id,map in enumerate([MAB_map,RMSB_map,Bias_map,Sampled_bias]):
    fig = plt.gcf()
    fig.set_size_inches(12,9.5)
    if map_id in [2,3]:
        plt.scatter([GNM_landonly[loc][5][1] for loc in range(4086)],[-GNM_landonly[loc][5][0] for loc in range(4086)],c=map,cmap='BrBG',s=24,marker='s',vmin=-np.max(np.abs(map)),vmax=np.max(np.abs(map)))
        plt.colorbar(location='bottom',fraction=0.0652,pad=0.001)
        plt.yticks([])
        plt.xticks([])
        plt.show()
        continue
    plt.scatter([GNM_landonly[loc][5][1] for loc in range(4086)],[-GNM_landonly[loc][5][0] for loc in range(4086)],c=map,s=24,marker='s',vmin=0)
    plt.colorbar(location='bottom',fraction=0.0652,pad=0.001)
    plt.yticks([])
    plt.xticks([])
    plt.show()

### Median VS Observed

In [ ]:
# median vs observed
binned_median_means = [np.log(np.mean(g_median_all[np.logical_and(all_target_rain.flatten()>-0.01+k*3, all_target_rain.flatten()<3+k*3     )])+1) for k in range(30)]
binned_median_std = [np.std(np.log(1+g_median_all[np.logical_and(all_target_rain.flatten()>-0.01+k*3, all_target_rain.flatten()<3+k*3     )])) for k in range(30)]

plt.plot(np.log([i for i in range(1,30*3,3)]),binned_median_means, color='red',linestyle='dashed',label='Mean/SD of pred in 3mm/day intervals')

for i,v in enumerate(range(1,30*3,3)):
    if i==0:
        continue
    plt.plot(np.log([v,v]),[binned_median_means[i]+binned_median_std[i] ,binned_median_means[i]-binned_median_std[i] ],'r',linewidth=1)

plt.plot([0,5],[0,5],linestyle='dashed',color='grey',label='y = x')
plt.scatter(np.log(all_target_rain.flatten()+1),np.log(g_median_all+1),marker='x',linewidths=0.3,alpha=0.3)

plt.axvline(x = np.log(10), color = 'purple', label = '10 mm rainfall',linewidth=1.3)

plt.legend(loc='upper left')

plt.xlabel('Observed log-trandformed rainfall')
plt.ylabel('Predicted log-trandformed rainfall')
plt.show()

### CRPS

In [ ]:
def estimate_CRPS_score_fast(obs,sims):
    '''
    obs: one vector of observations [y_1, y_2, ... y_n], ideally n corresponds to locations.
    sims: m vectors of simulations, each vector has length n. [[x_11, x_21,..., x_n1] ,..., [x_1m, x_2m,..., x_nm] ]

    returns: a vector [s1, ... , sn] of one score per location. this is for a single day, so would still need to average across days.
    '''
    m = len(sims)

    obs= np.array(obs)
    sims=np.array(sims)

    # |X-x|    
    diff_X_x = np.mean(
        np.abs(
            np.tile(obs,(m,1)) - sims
            ),
        axis=0)

    #|X-X'|
    diff_X_tilde_X = np.sum(
        np.abs(
            np.tile(sims,(m,1))-
            np.repeat(sims,m,axis=0)
            )
            ,axis=0
        )/(m*m-1)

    return 2*diff_X_x - diff_X_tilde_X 

    

In [ ]:
sims_all_copula = np.array(sims_all_copula) # 100,4087

In [ ]:
q_crps = []
for i,d in tqdm(enumerate(day_subset)):
    q_crps.append( estimate_CRPS_score_fast([GNM_landonly[l][3][d] for l in range(len(GNM_landonly))], sims_all_copula[i] ) )

In [ ]:
np.mean(q_crps)


In [ ]:
plt.scatter(range(500),np.mean(q_crps,axis=1))

In [ ]:
fig = plt.gcf() 
fig.set_size_inches(14,10)
plt.scatter([GNM_landonly[l][5][1] for l in range(len(GNM_landonly))],[-GNM_landonly[l][5][0] for l in range(len(GNM_landonly))],c=np.mean(q_crps,axis=0),s=27,marker='s')
plt.colorbar()
plt.title('mean CRPS per location')
plt.show()
for score in q_crps[:5]:
    plt.scatter([GNM_landonly[l][5][1] for l in range(len(GNM_landonly))],[-GNM_landonly[l][5][0] for l in range(len(GNM_landonly))],c=score,s=3)
    plt.colorbar()
    plt.title('crps per location for one sample')
    plt.show()

### Spatial Scoring Rule

In [ ]:
def Spatial_Energy_Score_single_day(beta,observations_y,simulations_Y):

    n = len(observations_y)
    m = len(simulations_Y)
    observations_y = np.array(observations_y)
    simulations_Y = np.array(simulations_Y)

    # First part |Y-y|. Gives the L2 dist scaled by power beta. Is a vector of length n/one value per location.
    diff_Y_y = np.power(
            np.linalg.norm(
                np.tile(observations_y,(m,1)) -
                simulations_Y,
            axis=1),
        beta)

    #Second part |Y-Y'|. 2* because pdist counts only once.
    diff_Y_Y = 2 * np.power(
        pdist(simulations_Y)
    ,beta)

    Energy = 2 * np.mean(diff_Y_y) - np.sum(diff_Y_Y)/(m*(m-1))
    return Energy

In [ ]:
Energy_diagnostic = []
for d in tqdm(range(500)):
    Energy_diagnostic.append(Spatial_Energy_Score_single_day(0.2,[GNM_landonly[l][3][day_subset[d]] for l in range(len(GNM_landonly))],sims_all_copula[i]))



In [ ]:
np.mean(Energy_diagnostic) 

In [ ]:
def Variogram_single_day(observations_y,simulations_Y,distance_matrix):

    n = len(observations_y)
    m = len(simulations_Y)
    y = np.array(observations_y)
    Y = np.array(simulations_Y)

    # inverse distance matrix, for weights w_ij:
    w_ij = np.nan_to_num( 1/distance_matrix, copy=False, nan=0, posinf=0, neginf=0)

    # |y_i - y_j| first part of Variogram
    diff_y_y = euclidean_distances(y.reshape(-1,1))

    # |Y_k,i - Y_k,j| second part of Variogram
    diff_Y_Y = np.mean(np.abs(np.tile(Y,n).reshape(m,n*n)-np.repeat(Y,n).reshape(m,n*n)),axis=0).reshape(n,n)

    Variogram = np.multiply(
        w_ij,
        np.power(
            diff_y_y - diff_Y_Y
        ,2)
    )

    return np.sum(Variogram)

In [ ]:
Variogram_diagnostic = []
for i,d in tqdm(enumerate(range(500))):
    Variogram_diagnostic.append(Variogram_single_day([GNM_landonly[l][3][day_subset[d]] for l in range(len(GNM_landonly))],sims_all_copula[i],dist_mat) )


In [ ]:
plt.plot(Variogram_diagnostic)
plt.plot([0,495],[np.mean(Variogram_diagnostic),np.mean(Variogram_diagnostic)])

In [ ]:
print(
    (np.sum(Variogram_diagnostic)-(np.sort(Variogram_diagnostic)[-20:]).sum())/len(Variogram_diagnostic)-20
    )

print(
    np.random.choice(Variogram_diagnostic, 20, replace=False).sum()/20
)

In [ ]:
np.mean(Variogram_diagnostic)

### Rank histogram

In [ ]:
#GNM_landonly[location][param][day]
# 0'pred_mu', 1'pred_disp', 2'target_did_rain', 3'target_rain_value', 4'pred_p', 5'location'

def get_cdf_gnm(day):
    p_times_cdf = np.multiply( #p*cdf(y)
        np.array([GNM_landonly[l][4][day] for l in range(len(GNM_landonly))]),
        scs.gamma(
            a=[1/GNM_landonly[l][1][day] for l in range(len(GNM_landonly))],
            scale=np.multiply([GNM_landonly[l][1][day] for l in range(len(GNM_landonly))],[GNM_landonly[l][0][day] for l in range(len(GNM_landonly))])
            ).cdf([GNM_landonly[l][3][day] for l in range(len(GNM_landonly))]))
    
    where_0 = np.argwhere(p_times_cdf==0).flatten()
    where_rain = np.argwhere(p_times_cdf>0).flatten()

    p_times_cdf[where_0] = np.random.uniform( # replace censored by u~[0,1-p]
        low=np.zeros(len(where_0)),
        high=(1-np.array([GNM_landonly[l][4][day] for l in range(len(GNM_landonly))])[where_0]).flatten()
        )
    
    p_times_cdf[where_rain] = (1-np.array([GNM_landonly[l][4][day] for l in range(len(GNM_landonly))]))[where_rain] + p_times_cdf[where_rain] # uncensored = (1-p) + p*cdf(y)

    return p_times_cdf

In [ ]:
obs_u_all = []
for d in tqdm(range(len(GNM_landonly[0][0]))):
    obs_u_all.append(get_cdf_gnm(d))

In [ ]:
obs_u_all = np.array(obs_u_all).flatten()
plt.hist(obs_u_all,bins=25,density=True,alpha=0.7)


In [ ]:
# Save
'''with open('obs_u_all_.txt','wb') as f:
    pickle.dump(obs_u_all_,f)
'''
#Load 
with open('obs_u_all_.txt','rb') as f:
    obs_u_all_ = pickle.load(f)

In [ ]:
plt.figure(figsize=(20,5))
obs_u_all = np.concatenate(np.array(obs_u_all_).flatten()).flatten()
plt.scatter( range(len(obs_u_all)),obs_u_all,s=0.000005)
plt.show()

In [ ]:
plt.figure(figsize=(5,6.5))
plt.hist(obs_u_all,bins=30,density=True,alpha=0.7)
plt.plot([0,1],[1,1])
plt.ylim(0,3)
plt.show()
plt.figure(figsize=(20,5))
plt.hist(obs_u_all,bins=100,density=True,alpha=0.7)
plt.plot([0,1],[1,1])   
plt.ylim(0,5)
plt.show()

In [ ]:
plt.hist(obs_u_all,bins=100,cumulative=True,alpha=0.7,align='mid')
plt.plot([0,1],[0,len(obs_u_all)],linestyle='dashed')
plt.show()
plt.figure(figsize=(20,5))
plt.hist(obs_u_all,bins=500,cumulative=True,alpha=0.7,align='mid',histtype='step',linewidth=2)
plt.plot([0,1],[0,len(obs_u_all)],linestyle='dashed')
plt.show()

### Correlation map

In [ ]:
with open('4y_sims.txt','rb') as f:
    sims_all_copula = pickle.load(f)

In [ ]:
with open('6y_sims.txt','rb') as f:
    sims_all_copula6 = pickle.load(f)
with open('8y_sims.txt','rb') as f:
    sims_all_copula8 = pickle.load(f)
with open('10y_sims.txt','rb') as f:
    sims_all_copula10 = pickle.load(f)

In [ ]:
with open('day_subset_robustness.txt','rb') as f:
    day_subset = pickle.load(f)

In [ ]:
for i,v in enumerate(np.array([GNM_landonly[l][5] for l in range(len(GNM_landonly))])): # closest to center 1107 [43 70]

    if math.dist(v,[48,76])<2:
        print(i,v)

In [ ]:
# observed corr
corr_mat = np.corrcoef( np.array([np.array([GNM_landonly[loc][3][day]for day in day_subset])for loc in range(len(GNM_landonly))]).flatten().reshape(len(GNM_landonly),len(day_subset)))
fig = plt.gcf() 
fig.set_size_inches(14,10)
plt.xlim(0,140)
plt.ylim(-100,0)
plt.scatter([GNM_landonly[loc][5][1] for loc in range(len(sim_test[0]))],[-GNM_landonly[loc][5][0] for loc in range(len(sim_test[0]))],c=corr_mat[:,3226      ],s=33,marker='s',vmin=0)
plt.colorbar(location='bottom',fraction=0.0652,pad=0.001)
plt.plot([76,76],[0,-100],color='black',linestyle='dotted',linewidth=1)
plt.plot([0,140],[-47,-47],color='black',linestyle='dotted',linewidth=1)
plt.yticks([])
plt.xticks([])
plt.tight_layout()
plt.savefig('obs_corr_4ytest.eps', format='eps')

plt.show()

In [ ]:
# sim corr
corr_mat = np.corrcoef(np.concatenate(np.array(sims_all_copula10)).transpose())
fig = plt.gcf() 
fig.set_size_inches(14,10)
plt.xlim(0,140)
plt.ylim(-100,0)
plt.scatter([GNM_landonly[loc][5][1] for loc in range(len(sim_test[0]))],[-GNM_landonly[loc][5][0] for loc in range(len(sim_test[0]))],c=corr_mat[:,3226    ],s=33,marker='s',vmin=0)
plt.colorbar(location='bottom',fraction=0.0652,pad=0.001)
plt.plot([76,76],[0,-100],color='black',linestyle='dotted',linewidth=1)
plt.plot([0,140],[-47,-47],color='black',linestyle='dotted',linewidth=1)
plt.yticks([])
plt.xticks([])
plt.tight_layout()
plt.savefig('sims_corr_4ytest.eps', format='eps')

plt.show()

In [ ]:
# median of sim corr
corr_mat = np.corrcoef(np.array(g_median_pred_all).transpose())
fig = plt.gcf() 
fig.set_size_inches(14,10)
plt.xlim(0,140)
plt.ylim(-100,0)
plt.scatter([GNM_landonly[loc][5][1] for loc in range(len(sim_test[0]))],[-GNM_landonly[loc][5][0] for loc in range(len(sim_test[0]))],c=corr_mat[:,108    ],s=27,marker='s',vmin=0)
plt.colorbar(location='bottom',fraction=0.0652,pad=0.001)
plt.plot([76,76],[0,-100],color='black',linestyle='dotted',linewidth=1)
plt.plot([0,140],[-47,-47],color='black',linestyle='dotted',linewidth=1)
plt.yticks([])
plt.xticks([])
plt.show()

### Plots comparing experiements:

In [ ]:
plt.plot([0,5,10,15,20,25],[0.8601,0.8352,0.8428,0.8560,0.8696,0.8824],marker='.',linewidth=3,markersize=10,label='4 year training')
plt.plot([0,5,10,15,20,25],[0.8617,0.8357,0.8428,0.8581,0.8734,0.8890],marker='.',linewidth=3,markersize=10,label='6 year training')
plt.plot([0,5,10,15,20,25],[0.8624,0.8356,0.8395,0.8503,0.8629,0.8756],marker='.',linewidth=3,markersize=10,label='8 year training')
plt.plot([0,5,10,15,20,25],[0.8676,0.8432,0.8493,0.8627,0.8764,0.8901],marker='.',linewidth=3,markersize=10,label='10 year training')
plt.ylim((0.5,0.9))
plt.legend()
fig = plt.gcf() 
fig.set_size_inches(10,7)
plt.tight_layout()
plt.ylabel('AUC')
plt.xlabel('Precipitation threshold (mm)')
plt.savefig('robustness_auc.eps', format='eps')
plt.show()

In [ ]:
rank_hist_20y = np.log([0.98161857, 0.97864251, 0.97957897, 0.98812185, 0.99906954,
        1.01136999, 1.02601243, 1.03284154, 1.04237115, 1.04817753,
        1.04794216, 1.0463021 , 1.04065403, 1.03421943, 1.02292831,
        1.01210542, 0.99817748, 0.98588038, 0.97085097, 0.95812836,
        0.94106939, 0.92277996, 0.90725969, 0.90099932, 1.11679787])
rank_hist_ConvCNP = np.log([1.0248582 , 1.02541557, 1.01611493, 1.01767681, 1.0191044 ,
        1.01836537, 1.02601098, 1.0297123 , 1.03591013, 1.04573709,
        1.05543907, 1.06261969, 1.06408996, 1.06217953, 1.05639312,
        1.04456025, 1.02403457, 1.0044901 , 0.98368496, 0.9558987 ,
        0.91061193, 0.86961335, 0.8297823 , 0.81821259, 0.99948386])
rank_hist_VAEGAN = np.log([6.93139183, 0.5285968 , 0.42249608, 0.38479105, 0.37226054, 0.36413387, 0.35319237, 0.34742652, 0.34565591, 0.34638231, 0.34872044, 0.36218165, 0.38218053, 0.39632275, 0.40476722, 0.42288198, 0.43212096, 0.4475117 , 0.47736245, 0.51023234, 0.56221579, 0.65403809, 0.76690348, 0.97772021, 7.45851312]
)
plt.plot(np.linspace(0,1,25),rank_hist_20y,linewidth=3,label='Cens-JGNM',alpha=1,zorder=5)
plt.plot(np.linspace(0,1,25),rank_hist_ConvCNP,linewidth=3,label='ConvCNP',alpha=1)
plt.plot(np.linspace(0,1,25),rank_hist_VAEGAN,linewidth=3,label='VAEGAN')
plt.plot([0,1],np.log([1,1]),color='black',linestyle='dashed',linewidth=1,zorder=-1)
plt.legend()
#plt.ylim(0,2)
fig = plt.gcf() 
fig.set_size_inches(14,10)
plt.tight_layout()
plt.ylabel('Frequency')
plt.xlabel('Rank of observed rain amongst forecast')
plt.savefig('combind_rank_hist_logged.eps', format='eps')
plt.show()


### IFS predictions

These are pointwise forecasts. Should use them as a baseline for pointwise metrics.

In [ ]:
IFS_2009 = np.fromfile('IFS_All_2009-07_2019-07_pred.dat', dtype="byte")
IFS_1999 = np.fromfile('IFS_All_1999-07_2019-07_pred.dat', dtype="byte")
IFS_1987 = np.fromfile('IFS_All_1987-07_2019-07_pred.dat', dtype="byte")

In [ ]:
print(len(IFS_2009),len(IFS_1999),len(IFS_1987))

### Comparing with marginal CP

In [ ]:
with open('day_subset_marginals.txt','rb') as f:
    day_subset = pickle.load(f)


In [ ]:
with open('x_y_CP_ecfd_500_subdays_samples_v0.txt','rb') as f:
    CP_ecfd_500_subdays_samples_v0 = pickle.load(f)
with open('x_y_CP_ecfd_500_subdays_samples_v1.txt','rb') as f:
    CP_ecfd_500_subdays_samples_v1 = pickle.load(f)
with open('x_y_CP_ecfd_500_subdays_samples_v3.txt','rb') as f:
    CP_ecfd_500_subdays_samples_v3 = pickle.load(f)

In [ ]:
with open('CP_rankhist_500_subdays_samples_v0.txt','rb') as f:
    CP_rankhist_500_subdays_samples_v0 = pickle.load(f)
with open('CP_rankhist_500_subdays_samples_v1.txt','rb') as f:
    CP_rankhist_500_subdays_samples_v1 = pickle.load(f)
with open('CP_rankhist_500_subdays_samples_v3.txt','rb') as f:
    CP_rankhist_500_subdays_samples_v3 = pickle.load(f)

In [ ]:
# P(rain>x) ~1m15 now. COMPARING WITH MARGINAL CP
from statsmodels.distributions.empirical_distribution import ECDF

target_ecdf = ECDF(np.array([np.array([GNM_landonly[loc][3][day] for loc in range(len(GNM_landonly))]) for day in tqdm(day_subset)]).flatten())
fig = plt.gcf() 
fig.set_size_inches(14,10)

gamma_sampled_ecdf = ECDF(np.array(sims_all_copula).flatten())

#gamma_sampled_ecdf = ECDF(np.array(sims_all_copula).flatten())
plt.plot(gamma_sampled_ecdf.x[::1000],1-gamma_sampled_ecdf.y[::1000],linewidth=5, zorder=5,label='Gamma-mixture')

#plt.plot(gamma_sampled_ecdf.x[::1000],1-gamma_sampled_ecdf.y[::1000],linestyle='--',linewidth=5, zorder=5,label='Sampled forecasts')
#plt.scatter(gamma_sampled_ecdf.x[::10000000],1-gamma_sampled_ecdf.y[::10000000],linestyle='dotted',color='cornflowerblue',marker='x',linewidth=20,s=3)
#plt.plot(CP_ecfd_500_subdays_samples_v0[0],CP_ecfd_500_subdays_samples_v0[1],linewidth=5,label='CP')
#plt.plot(CP_ecfd_500_subdays_samples_v1[0],CP_ecfd_500_subdays_samples_v1[1],linewidth=5,alpha=0.7,label='CP v1')
plt.plot(CP_ecfd_500_subdays_samples_v3[0],CP_ecfd_500_subdays_samples_v3[1],linewidth=5,alpha=0.7,label='CP')

plt.plot(target_ecdf.x,1-target_ecdf.y,linewidth=5,label='Target rainfall',color='black')
plt.xlabel('Precipitation (mm)')
plt.ylabel('Probability of precipitatoin > x')
plt.xlim(-1,25)
plt.legend()
plt.tight_layout()

plt.savefig('Marginal_gamma_CP_sampled_ecdf.eps', format='eps')


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# AUC
fig = plt.gcf() 
fig.set_size_inches(14,10)
plt.plot([0,5,10,15,20,25],[0.86695,0.84507,0.84203,0.82158,0.78247,0.732],marker='.',linewidth=5,markersize=20,label='Gamma-mixture')
#plt.plot([0,5,10,15,20,25],[0.64404,0.83492,0.79106,0.70592,0.61803,0.5716],marker='.',linewidth=5,markersize=20,label='CP v0')
#plt.plot([0,5,10,15,20,25],[0.64789,0.83601,0.78946,0.68983,0.59551,0.55476],marker='.',linewidth=5,markersize=20,label='CP v1')
plt.plot([0,5,10,15,20,25],[0.63384,0.8385,0.79,0.70024,0.61572,0.56913],marker='.',linewidth=5,markersize=20,label='CP')
plt.ylim((0.5,0.9))
plt.xlabel('Precipitation threshold (mm)')
plt.ylabel('AUC')
plt.legend()
plt.tight_layout()
plt.savefig('combind_AUC_marginal_gamma_vs_CP.eps', format='eps')
plt.show()

In [ ]:
# Comparing samples against observed to get ranks
sims_all_copula=np.array(sims_all_copula)
ranks = []
for d,day in tqdm(enumerate(day_subset)):
    ranks.append(np.mean((sims_all_copula[d,:,:]+np.random.uniform(low=-0.0001,high=0.0001,size=(100,4087)))<(np.array([GNM_landonly[l][3][day] for l in range(len(GNM_landonly))])+np.random.uniform(low=-0.0001,high=0.0001,size=4087)),axis=0))

ranks = np.array(ranks)

In [ ]:
np.random.uniform(low=-0.01,high=0.01,size=4087)

In [ ]:
test = CP_rankhist_500_subdays_samples_v3

In [ ]:
CP_rankhist_500_subdays_samples_v0[0],CP_rankhist_500_subdays_samples_v1[0],CP_rankhist_500_subdays_samples_v3[0]

In [ ]:
np.linspace(0,1,len(u_hist[0]))

In [ ]:
u_ranks = ranks.flatten()
u_hist = np.histogram(u_ranks,bins=25,density=True)

fig = plt.gcf() 
fig.set_size_inches(14,10)

plt.plot(np.linspace(0,1,25),u_hist[0][0:],linewidth=5,label='Gamma-mixture')
#plt.plot(np.linspace(0.04166667,1,24),CP_rankhist_500_subdays_samples_v0[1:],linewidth=5,label='CP',color=(1, 0.5, 0))
#plt.plot([0,0.04166667],[3.7,CP_rankhist_500_subdays_samples_v0[1]],linewidth=5,color=(1, 0.5, 0))
#plt.plot(0,3.7,marker='o',markersize=10,color=(1, 0.5, 0))
#plt.text(0.04166667,3.75,'CP: '+str(np.round(CP_rankhist_500_subdays_samples_v0[0],2)),fontsize=20)
#plt.plot(np.linspace(0.04166667,1,24),CP_rankhist_500_subdays_samples_v1[1:],linewidth=5,alpha=0.7,label='CP v1',color='green')
#plt.plot([0,0.04166667],[3.5,CP_rankhist_500_subdays_samples_v1[1]],linewidth=5,alpha=0.7,color='green')
#plt.plot(0,3.5,color='green',marker='o',markersize=10)
#plt.text(0.04166667,3.45,'CP v1: '+str(np.round(CP_rankhist_500_subdays_samples_v1[0],2)),fontsize=20)
plt.plot(np.linspace(0.04166667,1,24),CP_rankhist_500_subdays_samples_v3[1:],linewidth=5,label='CP v3',color=(1, 0.5, 0))
plt.plot([0,0.04166667],[3.6,CP_rankhist_500_subdays_samples_v3[1]],linewidth=5,color=(1, 0.5, 0))
plt.text(0.04166667,3.6,'CP: '+str(np.round(CP_rankhist_500_subdays_samples_v3[0],2)),fontsize=20)
plt.plot(0,3.6,color=(1, 0.5, 0),marker='o',markersize=10)

plt.plot([0,1],[1,1],color='black',linestyle='--')
plt.legend()
plt.ylim(0,4)
plt.xlabel('Frequency')
plt.ylabel('Rank of observed rain among forecast')
plt.tight_layout()
plt.savefig('rankhist_gamma_vs_CP.eps', format='eps')
